In [6]:
# Authors: Naveen Lalwani, Rangeesh Muthaiyan
# Script to prune the trained Deep Neural Networks

import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Sequential
from keras import layers
from keras import callbacks
from tensorflow.keras.datasets import cifar10

# External Library used for Pruning 
from kerassurgeon import identify
from kerassurgeon.operations import delete_channels

from keras.utils import np_utils
import time

In [2]:
training_verbosity = 2
# Download data if needed and import.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_train = np_utils.to_categorical(y_train)

y_test = np_utils.to_categorical(y_test)

dataset_train = tf.data.Dataset.from_tensor_slices((
    tf.cast(x_train/255, tf.float32),
    tf.cast(y_train, tf.int64)
)).shuffle(1000).batch(256)

dataset_test = tf.data.Dataset.from_tensor_slices((
    tf.cast(x_test/255, tf.float32),
    tf.cast(y_test, tf.int64)
)).batch(256)

In [3]:
# Create LeNet-5 model
model = Sequential()
model.add(Conv2D(6,
                 [5, 5],
                 input_shape=[32, 32, 3],
                 activation='relu',
                 name='conv_1'))
model.add(MaxPool2D())
model.add(Conv2D(16, [5, 5], activation='relu', name='conv_2'))
model.add(MaxPool2D())
#model.add(layers.Permute((2, 1, 3)))
model.add(Flatten())
model.add(Dense(120, activation='relu', name='dense_1'))
model.add(Dense(84, activation='relu', name='dense_2'))
model.add(Dense(10, activation='softmax', name='dense_3'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         min_delta=0,
                                         patience=10,
                                         verbose=training_verbosity,
                                         mode='auto')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                        factor=0.1,
                                        patience=5,
                                        verbose=training_verbosity,
                                        mode='auto',
                                        epsilon=0.001,
                                        cooldown=0,
                                        min_lr=0)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.
C:\Anaconda\lib\site-packages\keras\callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [4]:
# Train LeNet on MNIST
results = model.fit(x_train,
                    y_train,
                    epochs=200,
                    batch_size=256,
                    validation_data = (x_test, y_test),
                    callbacks=[early_stopping, reduce_lr]
                    )

model.save("./LeNet5_cifar10_results/before_pruning.h5")

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 50000 samples, validate on 10000 samples
Epoch 1/200
50000/50000 [==============================] - 68s 1ms/step - loss: 2.9223 - acc: 0.1016 - val_loss: 2.3024 - val_acc: 0.1007
Epoch 2/200
50000/50000 [==============================] - 29s 581us/step - loss: 2.2877 - acc: 0.1178 - val_loss: 2.2382 - val_acc: 0.1658
Epoch 3/200
50000/50000 [==============================] - 28s 557us/step - loss: 2.1053 - acc: 0.2330 - val_loss: 1.9033 - val_acc: 0.3095
Epoch 4/200
50000/50000 [==============================] - 33s 656us/step - loss: 1.7659 - acc: 0.3558 - val_loss: 1.6950 - val_acc: 0.3787
Epoch 5/200
50000/50000 [==============================] - 30s 599us/step - loss: 1.6267 - acc: 0.4053 - val_loss: 1.5791 - val_acc: 0.4215
Epoch 6/200
50000/50000 [==============================] - 23s 458us/step - loss: 1.5139 - acc: 0.4503 - val_loss: 1.5365 - val_acc: 0.4424
Epoch 7/200
50000/50000 [==============================] - 18s 359us/step - loss: 1.4245 - acc: 0.4853 - val_los

In [9]:
start = time.clock()
_, accuracy = model.evaluate(x_test, y_test, batch_size=256, verbose=2)
end = time.clock()
inferTime = end - start
print(f"Test Accuracy =  {accuracy:.4f}, Inference Time = {inferTime:.2f}s")

model.save("./LeNet5_cifar10_results/before_pruning.h5")

Test Accuracy: 0.5537, Inference Time: 1.61s


In [10]:
layer_name = 'dense_1'

while accuracy > 0.50:
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, x_test)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256)

    model.save("./LeNet5_cifar10_results/after_pruning.h5")
    print('model accuracy after pruning: ', accuracy, '\n')

    results = model.fit(x_train,
                        y_train,
                        epochs=100,
                        batch_size=256,
                        callbacks=[early_stopping, reduce_lr],
                        validation_data = (x_test, y_test))

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256,
                          verbose=2)
    model.save("./LeNet5_cifar10_results/after_pruning_retrain.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

Deleting 10/120 channels from layer: dense_1
10000/10000 [==============================] - 11s 1ms/step
model accuracy after pruning:  0.5532 

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 26s 526us/step - loss: 0.8890 - acc: 0.6835 - val_loss: 1.4267 - val_acc: 0.5374
Epoch 2/100
50000/50000 [==============================] - 17s 348us/step - loss: 0.8504 - acc: 0.6966 - val_loss: 1.4551 - val_acc: 0.5383
Epoch 3/100
50000/50000 [==============================] - 17s 343us/step - loss: 0.8180 - acc: 0.7086 - val_loss: 1.4864 - val_acc: 0.5383
Epoch 4/100
50000/50000 [==============================] - 18s 352us/step - loss: 0.8038 - acc: 0.7130 - val_loss: 1.4902 - val_acc: 0.5418
Epoch 5/100
50000/50000 [==============================] - 18s 357us/step - loss: 0.7700 - acc: 0.7259 - val_loss: 1.5287 - val_acc: 0.5389
Epoch 6/100
50000/50000 [==============================] - 19s 375us/step - loss: 0.7486 - acc: 0.7327 - 

In [12]:
start = time.clock()
_, accuracy = model.evaluate(x_test, y_test, batch_size=256, verbose=2)
end = time.clock()
inferTime = end - start
print(f"Test Accuracy: {accuracy:.4f}, Inference Time: {inferTime:.2f}s")

model.save("./LeNet5_cifar10_results/after_pruning_retrain.h5")

Test Accuracy: 0.4944, Inference Time: 1.88s


In [13]:
layer_name = 'dense_2'

while accuracy > 0.49:
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, x_test)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256)

    model.save("./LeNet5_cifar10_results/after_pruning_dense2.h5")
    print('model accuracy after pruning: ', accuracy, '\n')

    results = model.fit(x_train,
                        y_train,
                        epochs=100,
                        batch_size=256,
                        callbacks=[early_stopping, reduce_lr],
                        validation_data = (x_test, y_test))

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256,
                          verbose=2)
    model.save("./LeNet5_cifar10_results/after_pruning_retrain_dense2.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

Deleting 0/84 channels from layer: dense_2
10000/10000 [==============================] - 13s 1ms/step
model accuracy after pruning:  0.4944 

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 31s 624us/step - loss: 0.3300 - acc: 0.8830 - val_loss: 3.6493 - val_acc: 0.4941
Epoch 2/100
50000/50000 [==============================] - 20s 397us/step - loss: 0.3285 - acc: 0.8828 - val_loss: 3.5860 - val_acc: 0.4892
Epoch 3/100
50000/50000 [==============================] - 22s 431us/step - loss: 0.3048 - acc: 0.8895 - val_loss: 3.5816 - val_acc: 0.4930
Epoch 4/100
50000/50000 [==============================] - 25s 492us/step - loss: 0.3064 - acc: 0.8900 - val_loss: 3.6733 - val_acc: 0.4909
Epoch 5/100
50000/50000 [==============================] - 23s 457us/step - loss: 0.2959 - acc: 0.8928 - val_loss: 3.6236 - val_acc: 0.4918
Epoch 6/100
50000/50000 [==============================] - 21s 418us/step - loss: 0.3013 - acc: 0.8910 - va

In [14]:
t = time.clock()
l,accuracy = model.evaluate(x_test, y_test, batch_size=256, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {accuracy:.4f}, Timing: {t:.2f}s")

model.save("./LeNet5_cifar10_results/after_pruning_retrain_dense2.h5")

Test Accuracy: 0.4869, Timing: 1.57s


In [15]:
layer_name = 'conv_2'

while accuracy > 0.49:
    layer = model.get_layer(name=layer_name)
    apoz = identify.get_apoz(model, layer, x_test)
    high_apoz_channels = identify.high_apoz(apoz)
    model = delete_channels(model, layer, high_apoz_channels)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256)

    model.save("./LeNet5_cifar10_results/after_pruning_conv2.h5")
    print('model accuracy after pruning: ', accuracy, '\n')

    results = model.fit(x_train,
                        y_train,
                        epochs=100,
                        batch_size=256,
                        callbacks=[early_stopping, reduce_lr],
                        validation_data = (x_test, y_test))

    loss,accuracy = model.evaluate(x_test,
                          y_test,
                          batch_size=256,
                          verbose=2)
    model.save("./LeNet5_cifar10_results/after_pruning_retrain_conv2.h5")
    print('model accuracy after retraining: ', accuracy, '\n')

In [17]:
t = time.clock()
l,accuracy = model.evaluate(x_test, y_test, batch_size=256, verbose=2)
t = time.clock() - t
print(f"Test Accuracy: {accuracy:.4f}, Timing: {t:.2f}s")

model.save("./LeNet5_cifar10_results/after_pruning_retrain_conv2.h5")

Test Accuracy: 0.4869, Timing: 3.79s
